---
title: "Roulis Lab - Yale Colon Cancer Analysis I"
format:
  pdf:
    code-overflow: wrap
    geometry: margin=0.5in
---

# LOG FILE FOR YALE COLON CANCER ANALYSIS — PART I

**Author:** Bruno Ndiba Mbwaye Roy, School of Engineering and Applied Science (SEAS) 2027  
**Research mentor:** Manolis Roulis, Perelman School of Medicine (PSOM), Department of Pathology and Laboratory Medicine  
**Date:** February 19, 2026

---

## What this log covers

This notebook is the first part of our Yale colon cancer analysis log. We
take the raw data from the microscope and turn it into organized datasets,
maps of the cells, and figures that show where different cell types are and
which genes they use. Everything we create is saved in the **YALE Data**
folder; the original files stay in **Raw Data**.

**What we do, step by step:**

1. **Loading and organizing the raw data.** We start by loading the five raw
   files: the gene readings per cell, where each image region is, cell
   measurements, the transcript list, and the cell outlines. We then match
   each row of numbers to the right cell and put everything into one
   dataset. That dataset is saved so the rest of the notebook can use it.

2. **First look at the tissue.** We make one picture per image region (361
   in total). Each picture has two panels: on the left we show where the
   cells sit and color them by the vendor’s cell-type label; on the right we
   draw the same cells as outlines with the same colors. This gives a quick
   view of how cell types are arranged in the tissue.

3. **Cleaning the data and finding cell groups.** We remove low-quality
   cells, scale the gene readings so we can compare cells fairly, and then
   find groups of similar cells. We build a 2D “map” of the cells and
   assign each cell to a cluster. We save simple plots that show data
   quality, the map colored by cluster, and which genes stand out in each
   cluster.

4. **Summarizing the top genes per cluster.** For each cluster we list the
   top 20 genes that best define it. We save a table (yes/no per gene per
   cluster) and one small dot plot per cluster so we can see at a glance
   which genes are “on” in which group.

5. **Maps colored by each top gene.** We draw one map per gene for the top
   10 marker genes in each cluster. Each map is colored by how much that
   gene is turned on in each cell, so we see where that gene is active.

6. **Map colored by the vendor’s cell types.** We make one map where cells
   are colored by the vendor’s cell-type labels (not our clusters). That
   lets us compare our clusters with those labels.

7. **Where each cluster sits in the tissue.** We make the same kind of
   pictures as in step 2 (one per image region), but we color cells by our
   clusters. So we see where each cluster is located in real tissue.

8. **Maps for known cell-type genes.** We take a hand-picked list of genes
   that mark known cell types (epithelial, immune, stroma, etc.) and, for
   each gene in our data, we draw a map colored by that gene. So we see
   where those classic markers are expressed.

9. **Same genes with a cluster breakdown.** For those same marker genes we
   make a combined figure: the map on the left and, on the right, a bar
   chart showing what share of the cells that “have” that gene fall in each
   of our clusters, with bar color showing how strong the signal is. That
   helps us link our clusters to known cell types.

**Where everything is saved (YALE Data):**

| Folder | Contents |
|--------|----------|
| **YALE Data** (root) | `colon_adata.h5ad`; `colon_adata_clustered.h5ad`; see below. |
| **Plots (QC, UMAP & DOTPLOT)** | QC violins/scatter; PCA variance; UMAP Leiden; dotplot. |
| **fov_combined_plots/** | 361 FOV figures (InSituType scatter + polygons). |
| **leiden_markers/** | Top-20 CSV; dotplots per cluster (dotplots/). |
| **feature_plots/** | Per-cluster (Section 5) + per-celltype (Sections 8–9). |
| **feature_plots/per_celltype/** | Curated cell-type marker UMAP feature plots. |
| **feature_plots/per_celltype_with_cluster_hue/** | Same + cluster bar panels. |
| **umap_plots/** | UMAP colored by InSituType. |
| **fov_leiden_plots/** | Per-FOV Leiden scatter + polygons. |
| **Raw Data** (DATA_Bruno) | Source: expr matrix, metadata, FOV, tx, polygons. |

\newpage

---

## Important definitions

This section explains the main ideas and terms used in the notebook, in plain language. Where relevant, we mention the **function** used in the code (e.g. `sc.tl.umap`) so you can search for it if needed.

---

**UMAP (Uniform Manifold Approximation and Projection)**  
UMAP is a way to take many measurements per cell (thousands of genes) and represent them in a simple 2D picture. Cells that are similar in their gene activity end up close together on the plot; cells that are different end up far apart. So a UMAP is like a *map* of cells based on how alike they are. In the code we use **`sc.tl.umap`** to build this map. When we say "expression UMAP," we mean a UMAP built from gene expression (how much each gene is turned on in each cell).

---

**Clustering (e.g. Leiden clustering)**  
Clustering groups cells that look similar into the same "cluster." For example, we might get clusters that correspond to fibroblasts, muscle cells, immune cells, etc. **Leiden** is the name of the method we use to find these groups; it uses the connections between nearby cells (from the UMAP or from gene space) to decide who belongs together. In the code we use **`sc.tl.leiden`**. A **subcluster** is a finer grouping *within* a bigger group (e.g. different types of fibroblasts within all fibroblast-like cells). We get subclusters by running clustering again on just that subset of cells.

---

**Expression**  
Expression means how much a gene is "on" in a cell—i.e. how much RNA or protein is produced. When we color a UMAP or a spatial plot by a gene, we are showing where that gene is more or less active across cells or across the tissue.

---

**Spatial plot**  
A spatial plot shows cells at their *real* positions in the tissue (x and y coordinates from the microscope). So unlike a UMAP (which is a "similarity map"), a spatial plot shows *where* cells actually sit in the slice—useful to see if a cell type or a gene is in a particular region (e.g. near the muscle layer or the lining).

---

**Slice**  
A slice is one piece of tissue that was imaged (e.g. one section from the colon). We have multiple slices per sample; each has its own x, y coordinates. Many analyses are done *per slice* so that we only compare neighbors within the same physical tissue.

---

**Neighborhood**  
In this notebook, "neighborhood" can mean two things: (1) The **spatial** neighbors of a cell—the *k* closest cells in the tissue (by x, y distance). (2) A **cluster** from an analysis that uses the *composition* of those neighbors (e.g. how many Tier1/Tier3 types are nearby). So "neighborhood composition" is: for each cell, what types of cells are around it, and in what proportion?

---

**Marker genes / top markers**  
Marker genes are genes that are especially characteristic of a cell type or cluster (e.g. high in that group and lower elsewhere). We find them by comparing gene expression between clusters; the "top" markers are the ones that best distinguish a given cluster. In the code we use **`sc.tl.rank_genes_groups`** to rank genes and **`sc.get.rank_genes_groups_df`** to export the list.

---

**Dot plot**  
A dot plot shows how strongly marker genes are expressed in each cluster: dot *size* usually means "how many cells in that cluster express the gene," and dot *color* means "how strong the expression is on average." So you can quickly see which genes are specific to which cluster.

---

**Feature plot**  
A feature plot is a UMAP (or sometimes a spatial plot) where each point (cell) is colored by a single "feature"—usually the expression level of one gene. So you see *where* in the map or in the tissue that gene is high or low.

---

**PCA (Principal Component Analysis)**  
PCA is a way to reduce many variables (e.g. thousands of genes) to a smaller set of "summary" dimensions that capture most of the variation. We often run PCA first and then build the UMAP or the neighbor graph from these summary dimensions, which makes the analysis faster and less noisy. In the code we use **`sc.tl.pca`**.

---

**k nearest neighbors (kNN)**  
For each cell, we look at the *k* "nearest" other cells—nearest either in gene space (similar expression) or in space (physical distance in the tissue). *k* is just a number we choose (e.g. 15 or 40). The neighbor graph (used for clustering and UMAP) is built from these connections; **`sc.pp.neighbors`** does this in the code.

## REPRODUCING YALE COLON TMA PLOTS

- All outputs are saved to **DATA_Bruno → YALE Data**.
- Raw input files live in **DATA_Bruno → YALE Data → Raw Data**.
- Run section cells in order; data paths assume files in working dir or paths set below.

## 1. Yale Colon TMA — Data loading and AnnData creation

### Location of originals: DATA_Bruno → YALE Data (outputs); Raw Data (inputs)

**Purpose.** Turn the Yale Colon TMA flat files into a single AnnData object that
downstream sections (QC, UMAP, Leiden, plots) can use. AnnData holds cells as
rows, genes as columns, plus cell metadata and spatial info.

**Process (reproduce in order).**
1. Load the five raw files from **Raw Data** (see list below).
2. Inspect shapes and columns so you know what each table contains.
3. Align expression to cell IDs: expression matrix is keyed by (fov, cell_ID);
   the transcript file gives the mapping to string IDs like `c_1_1_1`. We merge
   to attach that ID to every expression row.
4. Build the AnnData: `X` = gene counts (sparse), `obs` = cell metadata (one
   row per cell, index = cell ID), `var` = gene names. Then attach spatial
   coordinates (cell centers in local pixels), FOV positions, polygon
   outlines, and a short dataset summary in `uns`.
5. Save as `colon_adata.h5ad` and upload to **YALE Data**.

**Input files (Raw Data):**
- `Colon_TMA_exprMat_file.csv.gz`: counts per (fov, cell_ID) × gene.
- `Colon_TMA_fov_positions_file.csv.gz`: FOV id and global px/mm positions.
- `Colon_TMA_metadata_file.csv.gz`: one row per cell; QC and annotations.
- `Colon_TMA_tx_file.csv.gz`: per-transcript info; used for (fov, cell_ID)→cell.
- `Colon_TMA-polygons.csv.gz`: polygon vertices per cell for drawing outlines.

**Output (YALE Data):** `colon_adata.h5ad` — one file containing expression,
obs, var, obsm["spatial"], uns["fov_positions"], uns["polygons"], uns["dataset_info"].

Place raw files in the working directory (or set paths in the first code cell).
Run cells in order.

In [ ]:
# Imports — run once (pandas/numpy: tables; scanpy/anndata: single-cell; scipy: sparse)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import patches as mpatches
import seaborn as sns
import scanpy as sc
import anndata as ad
from scipy import sparse
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")
sc.settings.verbosity = 3  # scanpy progress messages
print("Libraries imported successfully!")

In [ ]:
# Load the five raw files (paths: working dir or Raw Data)
# expr_mat index = (fov, cell_ID); columns = gene names
print("Loading data files...")
expr_mat = pd.read_csv(
    "Colon_TMA_exprMat_file.csv.gz", compression="gzip", index_col=0
)
fov_positions = pd.read_csv(
    "Colon_TMA_fov_positions_file.csv.gz", compression="gzip"
)
metadata = pd.read_csv(
    "Colon_TMA_metadata_file.csv.gz", compression="gzip"
)
tx_file = pd.read_csv(
    "Colon_TMA_tx_file.csv.gz", compression="gzip"
)
polygons = pd.read_csv(
    "Colon_TMA-polygons.csv.gz", compression="gzip"
)
print(f"Expression matrix: {expr_mat.shape}")
print(f"FOV positions: {fov_positions.shape}, Metadata: {metadata.shape}")
print(f"Transcript file: {tx_file.shape}, Polygons: {polygons.shape}")
print("Data loading completed!")

In [ ]:
# Explore structure
print("=== EXPRESSION MATRIX ===")
print(f"Shape: {expr_mat.shape}")
print(f"Index sample: {expr_mat.index[:5].tolist()}...")
print(f"Columns (genes) sample: {expr_mat.columns[:5].tolist()}...")
print("\n=== FOV POSITIONS ===")
print(fov_positions.head())
print("\n=== METADATA ===")
print(metadata.head())
print("\nColumns:", metadata.columns.tolist())

In [ ]:
# Transcript and polygon structure
print("=== TRANSCRIPT FILE ===")
print(tx_file.head())
print("Columns:", tx_file.columns.tolist())
print("\n=== POLYGONS ===")
print(polygons.head())
print("Columns:", polygons.columns.tolist())

In [ ]:
# Map (fov, cell_ID) to string cell id (e.g. c_1_1_1) for AnnData index
expr = expr_mat.reset_index()  # fov, cell_ID become columns
cell_map = (
    tx_file[["fov", "cell_ID", "cell"]].drop_duplicates()
)
print(cell_map.head())
print(f"Mapping rows: {len(cell_map)}")

In [ ]:
# Attach cell id to each expression row (one-to-one)
expr_with_cell = expr.merge(
    cell_map, on=["fov", "cell_ID"], how="left", validate="one_to_one"
)
print(expr_with_cell[["fov", "cell_ID", "cell"]].head())
print("Rows without cell label:", expr_with_cell["cell"].isna().sum())

In [ ]:
# Build count matrix X: rows = cells (same order as expr_with_cell), cols = genes
non_gene_cols = ["fov", "cell_ID", "cell"]
gene_cols = [c for c in expr_with_cell.columns if c not in non_gene_cols]
print("Number of genes:", len(gene_cols))
# Sparse saves memory (many zeros)
X = sparse.csr_matrix(
    (expr_with_cell[gene_cols].astype("uint16")).values
)
print("Sparse X shape:", X.shape)

In [ ]:
# obs: one row per cell, index = cell id; merge in full metadata from vendor
obs = expr_with_cell[["cell", "fov"]].copy()
obs.index = obs["cell"].astype(str)
obs = obs.drop(columns=["cell"])
meta = metadata.copy()
meta.index = meta["cell"].astype(str)
overlap = [c for c in meta.columns if c in obs.columns]
meta = meta.drop(columns=overlap)
obs = obs.join(meta, how="left")
print(obs.head())
print("obs shape:", obs.shape)

In [ ]:
# var = gene names (rows of var = columns of X); build AnnData
var = pd.DataFrame(index=gene_cols)
var.index.name = "gene_id"
adata = ad.AnnData(X=X, obs=obs, var=var)
print(adata)

In [ ]:
# Attach spatial coords (cell centers), FOV table, polygons, and a short summary
coord_cols = ["CenterX_local_px", "CenterY_local_px"]
adata.obsm["spatial"] = adata.obs[coord_cols].values
adata.uns["fov_positions"] = fov_positions
adata.uns["polygons"] = polygons
adata.uns["dataset_info"] = {
    "expression_matrix_n_cells": X.shape[0],
    "expression_matrix_n_genes": X.shape[1],
    "n_metadata_columns": obs.shape[1],
    "n_polygon_vertices": len(polygons),
    "n_transcript_rows": len(tx_file),
    "source_files": {
        "expr_mat": "Colon_TMA_exprMat_file.csv.gz",
        "metadata": "Colon_TMA_metadata_file.csv.gz",
        "tx_file": "Colon_TMA_tx_file.csv.gz",
        "polygons": "Colon_TMA-polygons.csv.gz",
        "fov_positions": "Colon_TMA_fov_positions_file.csv.gz",
    },
}
print(adata)

In [ ]:
# Write AnnData to disk; upload this file to YALE Data on Box
adata.write("colon_adata.h5ad")

## 2. Initial visualizations — FOV combined plots

### Location of originals: DATA_Bruno → YALE Data → fov_combined_plots

**Purpose.** Visualize each field of view (FOV) with vendor cell-type labels
(InSituType). An FOV is one imaging region; there are 361 FOVs. Each figure has
two panels so you can compare point-based vs outline-based views.

**What each figure shows.**
- **Left panel:** Scatter of cell centers in local pixel coordinates
  (CenterX_local_px, CenterY_local_px). Each point is one cell, colored by
  InSituType cluster (a–j). Lets you see which cell types sit where in the FOV.
- **Right panel:** Same FOV drawn as polygon outlines (one outline per cell),
  with the same InSituType colors. Useful to see cell shapes and boundaries.

**Output.** One PNG per FOV: `fov_combined_plots/fov_001_combined.png` through
`fov_361_combined.png`. Upload the folder to **YALE Data**.

**Data.** Run Section 1 first, or load `adata = ad.read_h5ad('colon_adata.h5ad')`.
The first code cell below creates `fov_combined_plots`; then run the plot cells.

In [ ]:
# Optional: load adata if kernel was restarted
# adata = ad.read_h5ad("colon_adata.h5ad")
from matplotlib.colors import to_hex
plot_dir = Path("fov_combined_plots")
plot_dir.mkdir(exist_ok=True)
# Vendor InSituType cluster column (letters a–j)
cluster_col = "RNA_RNA.QC.Module_Cell.Typing.InSituType.1_1_clusters"
cluster_categories = (
    adata.obs[cluster_col].astype("category").cat.categories
)
# Stable colors across FOVs
palette = sns.color_palette("tab20", n_colors=max(len(cluster_categories), 1))
cluster_color_lookup = {
    cat: to_hex(c) for cat, c in zip(cluster_categories, palette)
}
print(f"Saving to: {plot_dir.resolve()}")
print(f"Cluster labels in '{cluster_col}': {len(cluster_categories)}")

In [ ]:
def make_fov_combined_plot(fov_id: int):
    """One figure: left = InSituType scatter, right = same FOV as polygons."""
    adata_fov = adata[adata.obs["fov"] == fov_id].copy()
    if adata_fov.n_obs == 0:
        raise ValueError(f"FOV {fov_id} has no cells.")
    x = adata_fov.obsm["spatial"][:, 0]
    y = adata_fov.obsm["spatial"][:, 1]
    cluster_series = (
        adata_fov.obs[cluster_col]
        .astype("category")
        .cat.remove_unused_categories()
    )
    scatter_colors = (
        cluster_series.map(cluster_color_lookup).astype(object).fillna("#333333")
    )
    cell_to_cluster = cluster_series.to_dict()
    fig, axes = plt.subplots(1, 2, figsize=(12, 6), sharey=True)
    axes[0].scatter(x, y, s=4, c=scatter_colors)
    axes[0].set_title(f"FOV {fov_id} — Cluster scatter")
    axes[0].set_xlabel("CenterX_local_px")
    axes[0].set_ylabel("CenterY_local_px")
    axes[0].invert_yaxis()
    legend_handles = [
        mpatches.Patch(color=cluster_color_lookup[cat], label=str(cat))
        for cat in cluster_series.cat.categories
        if cat in cluster_color_lookup
    ]
    if legend_handles:
        axes[0].legend(
            handles=legend_handles,
            title="InSituType",
            loc="upper left",
            bbox_to_anchor=(1.02, 1.0),
            borderaxespad=0.0,
        )
    poly = adata.uns.get("polygons")
    axes[1].set_title(f"FOV {fov_id} — Polygons")
    axes[1].set_xlabel("x_local_px")
    axes[1].set_ylabel("y_local_px")
    if poly is not None:
        poly_fov = poly[poly["fov"] == fov_id]
        for cell_label, df in poly_fov.groupby("cell"):
            cl = cell_to_cluster.get(cell_label)
            color = cluster_color_lookup.get(cl, "#777777")
            axes[1].plot(
                df["x_local_px"], df["y_local_px"],
                linewidth=0.5, color=color,
            )
    else:
        axes[1].text(0.5, 0.5, "No polygon data", ha="center", va="center")
    axes[1].invert_yaxis()
    axes[1].set_aspect("equal", adjustable="box")
    axes[0].set_aspect("equal", adjustable="box")
    fig.suptitle(f"FOV {fov_id}: Clusters & Polygons", fontsize=14)
    fig.tight_layout()
    return fig

In [ ]:
# Generate and save one figure per FOV (upload folder to YALE Data)
saved_files = []
for raw_fov in sorted(adata.obs["fov"].unique()):
    fov_id = int(raw_fov)
    fig = make_fov_combined_plot(fov_id)
    out_path = plot_dir / f"fov_{fov_id:03d}_combined.png"
    fig.savefig(out_path, dpi=250, bbox_inches="tight")
    plt.close(fig)
    saved_files.append(out_path)
print(f"Saved {len(saved_files)} figures to {plot_dir.resolve()}")

## 3. QC, normalization, embedding, and Leiden clustering

### Location of originals: DATA_Bruno → YALE Data → Plots (QC, UMAP & DOTPLOT)

**Purpose.** Filter low-quality cells, normalize and log-transform, find
highly variable genes, reduce dimension (PCA), build a neighbor graph, run
UMAP and Leiden clustering, and detect marker genes. All plots from this
section go into one folder for upload to **YALE Data → Plots (QC, UMAP &
DOTPLOT)**.

**Process (order matters).**
- **3.1–3.2:** Check QC columns and plot their distributions (violins +
  scatter). Lets you see count/feature/propNegative before filtering.
- **3.3:** Keep cells with nCount_RNA and nFeature_RNA ≥ 5th percentile.
- **3.4:** Normalize to 10k counts/cell, log1p; store raw in adata.raw.
- **3.5:** Select 3000 HVGs (Seurat v3), subset adata, scale (zero_center, cap 10).
- **3.6:** Fit PCA on up to 100k cells, project all; plot variance ratio.
- **3.7:** kNN (15, 15 PCs) → UMAP → Leiden (res. 0.8) → rank_genes_groups
  (Wilcoxon). Save UMAP colored by Leiden and one dotplot (top 5 genes per
  cluster) as overview.

**Outputs.** `Plots_QC_UMAP_DOTPLOT/`: qc_violins.png, qc_scatter.png,
pca_variance_ratio.png, umap_leiden.png, rank_genes_dotplot.png.

**Data.** `colon_adata.h5ad` from Section 1 (or load in first code cell).

In [ ]:
# Imports and paths; load adata (run if starting from Section 3)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import patches as mpatches
from matplotlib.colors import LinearSegmentedColormap, to_hex
import seaborn as sns
import scanpy as sc
import anndata as ad
from scipy import sparse
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")
sc.settings.verbosity = 3
# All Section 3 figures saved here; upload folder to YALE Data
plot_out = Path("Plots_QC_UMAP_DOTPLOT")
plot_out.mkdir(exist_ok=True)
adata = ad.read_h5ad("colon_adata.h5ad")
print(adata)

### 3.1 Quality-control overview

Confirm that vendor QC columns are in `adata.obs` and print summary stats
(count, mean, std, percentiles) for nCount_RNA, nFeature_RNA, Area,
Area.um2, propNegative. Use this to interpret the next plots.

In [ ]:
# Check QC columns exist; describe() gives count, mean, std, min, percentiles
qc_cols = {"nCount_RNA", "nFeature_RNA", "Area", "Area.um2", "propNegative"}
print(qc_cols <= set(adata.obs.columns))
print(adata.obs[list(qc_cols)].describe().T)

### 3.2 QC distribution plots

**Violins:** One panel per QC metric; each shows distribution across all cells
(jittered points). **Scatter:** nCount_RNA vs nFeature_RNA, points colored by
propNegative (high = more negative probe signal). Saved to plot_out.

In [ ]:
qc_plot_cols = ["nCount_RNA", "nFeature_RNA", "propNegative"]
sc.pl.violin(
    adata,
    keys=qc_plot_cols,
    jitter=0.4,
    multi_panel=True,
    rotation=45,
    show=False,
)
plt.gcf().savefig(plot_out / "qc_violins.png", dpi=200, bbox_inches="tight")
plt.close()

In [ ]:
sc.pl.scatter(
    adata,
    x="nCount_RNA",
    y="nFeature_RNA",
    color="propNegative",
    show=False,
)
plt.gcf().savefig(plot_out / "qc_scatter.png", dpi=200, bbox_inches="tight")
plt.close()

### 3.3 Basic QC thresholds

Keep only cells with nCount_RNA ≥ 5th percentile and nFeature_RNA ≥ 5th
percentile. Remaining cells are used for normalization and downstream steps.

In [ ]:
min_counts = adata.obs["nCount_RNA"].quantile(0.05)
min_features = adata.obs["nFeature_RNA"].quantile(0.05)
qc_mask = (
    (adata.obs["nCount_RNA"] >= min_counts)
    & (adata.obs["nFeature_RNA"] >= min_features)
)
print(f"Cells before: {adata.n_obs:,}")
adata = adata[qc_mask].copy()
print(f"Cells after: {adata.n_obs:,}")

### 3.4 Normalization and log transform

Scale each cell to total 10k counts; then log1p (log(1+x)). adata.raw keeps a
copy of this normalized data for later (e.g. plotting gene expression).

In [ ]:
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
adata.raw = adata

### 3.5 Highly variable genes and scaling

Select 3000 highly variable genes (Seurat v3). Subset adata to these genes;
then z-scale (zero mean, unit variance, capped at ±10) for PCA/neighbors.

In [ ]:
sc.pp.highly_variable_genes(
    adata, flavor="seurat_v3", n_top_genes=3000,
)
adata = adata[:, adata.var["highly_variable"]].copy()
sc.pp.scale(adata, zero_center=True, max_value=10)
print(f"Using {adata.n_vars} highly variable genes")

### 3.6 Dimensionality reduction (PCA)

Fit 50-component PCA on a random subset (up to 100k cells) for speed; then
project *all* cells. Variance-ratio plot shows how much variance each PC
captures (log scale). Stored in adata.obsm["X_pca"], adata.varm["PCs"].

In [ ]:
from sklearn.decomposition import PCA
# Subsample for speed; same seed for reproducibility
sample_size = min(100000, adata.n_obs)
rng = np.random.default_rng(0)
sample_idx = rng.choice(adata.n_obs, sample_size, replace=False)
adata_sample = adata[sample_idx].copy()
print(f"Fitting PCA on {adata_sample.n_obs:,} cells")
pca_model = PCA(n_components=50, svd_solver="auto", random_state=0)
pca_model.fit(adata_sample.X)

In [ ]:
# Project all cells (not just sample) into PC space
adata.obsm["X_pca"] = pca_model.transform(adata.X)
adata.varm["PCs"] = pca_model.components_.T
adata.uns["pca"] = {
    "variance": pca_model.explained_variance_,
    "variance_ratio": pca_model.explained_variance_ratio_,
    "params": {"n_comps": 50, "sample_size": sample_size},
}
plt.figure()
plt.plot(
    np.arange(1, 51),
    pca_model.explained_variance_ratio_,
    marker="o",
)
plt.xlabel("PC")
plt.ylabel("Variance ratio")
plt.title("PCA variance ratio (sample fit)")
plt.yscale("log")
plt.savefig(plot_out / "pca_variance_ratio.png", dpi=200, bbox_inches="tight")
plt.close()

### 3.7 Neighborhood graph, UMAP, Leiden, marker detection

Build kNN graph from first 15 PCs (15 neighbors); run UMAP and Leiden
(resolution 0.8). Then rank genes per Leiden cluster (Wilcoxon). Save:
umap_leiden.png (cells colored by cluster) and rank_genes_dotplot.png (top 5
genes per cluster; dot size = fraction expressing, color = mean expression).

In [ ]:
# kNN on 15 PCs, 15 neighbors; then UMAP and Leiden
sc.pp.neighbors(adata, n_neighbors=15, n_pcs=15)
sc.tl.umap(adata)
sc.tl.leiden(adata, resolution=0.8, key_added="leiden")

In [ ]:
sc.pl.umap(adata, color="leiden", legend_loc="on data", show=False)
plt.gcf().savefig(plot_out / "umap_leiden.png", dpi=250, bbox_inches="tight")
plt.close()

In [ ]:
sc.tl.rank_genes_groups(
    adata, groupby="leiden", method="wilcoxon",
)
sc.pl.rank_genes_groups_dotplot(
    adata, n_genes=5, groupby="leiden", show=False,
)
plt.gcf().savefig(
    plot_out / "rank_genes_dotplot.png", dpi=200, bbox_inches="tight"
)
plt.close()
print(f"Section 3 plots saved to {plot_out.resolve()}")

## 4. Export top-marker summaries

### Location of originals: DATA_Bruno → YALE Data → leiden_markers

**Purpose.** For each Leiden cluster, get the top 20 marker genes (from
rank_genes_groups in Section 3) and write (1) a CSV table and (2) one
dotplot per cluster for quick visual review.

**CSV (`leiden_top20_markers.csv`).** Rows = genes that appear in any
cluster’s top 20; columns = Leiden cluster ids. Each cell is "yes" or "no":
"yes" means that gene is in that cluster’s top 20 markers. Lets you see
overlap and specificity across clusters.

**Per-cluster dotplot (`dotplots/cluster_X_top20_dotplot.png`).** One image
per Leiden cluster. Each plot shows: *x-axis* = all Leiden clusters (0, 1,
2, …); *y-axis* = the top 20 marker genes for the *focal* cluster (the one
in the filename). For each gene and each cluster, a dot: *size* = fraction
of cells in that cluster expressing the gene, *color* = mean expression
(warmer = higher). So you see how specific the focal cluster’s markers are:
they should be large/warm in the focal cluster and smaller/cooler elsewhere.

**Output folder.** `leiden_markers/` (CSV + `dotplots/` subfolder). Uploaded to
**YALE Data**.

In [ ]:
# Output dirs: CSV and one dotplot per cluster
output_dir = Path("leiden_markers")
dotplot_dir = output_dir / "dotplots"
output_dir.mkdir(exist_ok=True)
dotplot_dir.mkdir(exist_ok=True)
clusters = adata.obs["leiden"].astype("category").cat.categories
print(f"Top markers for {len(clusters)} clusters -> {output_dir.resolve()}")

In [ ]:
# Dendrogram used by some scanpy plotters; we need it for consistency
sc.tl.dendrogram(adata, groupby="leiden")
top_n = 20
cluster_to_genes = {}
all_markers = set()
# Top 20 genes per cluster by log fold-change (vs rest)
for cluster in clusters:
    df = sc.get.rank_genes_groups_df(adata, group=str(cluster))
    top_genes = (
        df.sort_values("logfoldchanges", ascending=False)
        .head(top_n)["names"]
        .tolist()
    )
    cluster_to_genes[cluster] = top_genes
    all_markers.update(top_genes)
# Rows = genes, cols = clusters; "yes" if gene in that cluster's top 20
marker_table = pd.DataFrame("no", index=sorted(all_markers), columns=clusters)
for cluster, genes in cluster_to_genes.items():
    marker_table.loc[genes, cluster] = "yes"
csv_path = output_dir / "leiden_top20_markers.csv"
marker_table.to_csv(csv_path)
print(f"Saved {csv_path}")

In [ ]:
# One dotplot per cluster: x=all clusters, y=this cluster's top 20 genes
for cluster in clusters:
    sc.pl.rank_genes_groups_dotplot(
        adata,
        groupby="leiden",
        groups=[cluster],  # highlight this cluster
        n_genes=top_n,
        dendrogram=False,
        figsize=(6, 6),
        log=True,
        show=False,
        colorbar_title="avg expr",
    )
    fig = plt.gcf()
    fig.suptitle(f"Leiden {cluster} — top {top_n} markers", y=1.02)
    plot_path = dotplot_dir / f"cluster_{cluster}_top{top_n}_dotplot.png"
    fig.savefig(plot_path, dpi=200, bbox_inches="tight")
    plt.close(fig)
print(f"Saved {len(clusters)} dotplots to {dotplot_dir}")

## 5. Feature plots for top Leiden markers

### Location of originals: DATA_Bruno → YALE Data → feature_plots

**Purpose.** For each of the top marker genes (from Section 4), draw a UMAP
with cells colored by that gene’s expression. Lets you see *where* in the
embedding (and thus which clusters/regions) express the gene.

**What each plot is.** One UMAP, one gene. Each point = one cell; color =
expression level of the gene (blue = low, green → yellow → red = high).
vmax="p99" caps the color scale at the 99th percentile so a few high cells
don’t wash out the rest. Title: "Cluster X marker: GENE".

**Layout.** `feature_plots/per_cluster/cluster_X/` contains one PNG per gene
(GENE_feature.png) for the top 10 markers of cluster X. Uploaded to **YALE Data**.

In [ ]:
# Custom colormap: low expression = blue, high = red
colors = ["blue", "green", "yellow", "red"]
custom_cmap = LinearSegmentedColormap.from_list(
    "BlueGreenYellowRed", colors, N=256,
)
feature_plot_root = Path("feature_plots")
cluster_feature_dir = feature_plot_root / "per_cluster"
feature_plot_root.mkdir(exist_ok=True)
cluster_feature_dir.mkdir(exist_ok=True)
n_markers_per_cluster = 10  # top 10 genes per cluster
plots_created = 0

In [ ]:
# One UMAP per gene (top n_markers_per_cluster per cluster)
for cluster in clusters:
    genes = cluster_to_genes.get(cluster, [])[:n_markers_per_cluster]
    if not genes:
        continue
    cluster_dir = cluster_feature_dir / f"cluster_{cluster}"
    cluster_dir.mkdir(exist_ok=True)
    for gene in genes:
        if gene not in adata.var_names:
            continue
        sc.pl.umap(
            adata,
            color=gene,
            cmap=custom_cmap,
            size=1.5,
            vmax="p99",
            title=f"Cluster {cluster} marker: {gene}",
            show=False,
        )
        fig = plt.gcf()
        fig.savefig(
            cluster_dir / f"{gene}_feature.png",
            dpi=200,
            bbox_inches="tight",
        )
        plt.close(fig)
        plots_created += 1
print(f"Saved {plots_created} feature plots to {cluster_feature_dir}")

## 6. UMAP colored by vendor InSituType clusters

### Location of originals: DATA_Bruno → YALE Data → umap_plots

**Purpose.** Compare our Leiden embedding with the vendor’s cell-type labels
(InSituType). One UMAP, cells colored by InSituType cluster (a–j).

**What the plot shows.** Same 2D UMAP as in Section 3, but colors = vendor
InSituType instead of Leiden. Lets you see whether Leiden clusters align
with or cut across vendor types. Legend on the right margin.

**Output.** `umap_plots/umap_insitutype_clusters.png`. Uploaded to **YALE Data**.

In [ ]:
# Vendor InSituType column (used in Section 2 as well)
cluster_col = "RNA_RNA.QC.Module_Cell.Typing.InSituType.1_1_clusters"
cluster_categories = (
    adata.obs[cluster_col].astype("category").cat.categories
)
cluster_palette = sns.color_palette(
    "tab20", n_colors=max(len(cluster_categories), 1)
)
cluster_colors = [to_hex(c) for c in cluster_palette]
cluster_color_lookup = dict(zip(cluster_categories, cluster_colors))
umap_dir = Path("umap_plots")
umap_dir.mkdir(exist_ok=True)

In [ ]:
sc.pl.umap(
    adata,
    color=cluster_col,
    palette=cluster_colors,
    legend_loc="right margin",
    size=1.5,
    title="UMAP colored by InSituType clusters",
    show=False,
)
fig = plt.gcf()
fig.savefig(
    umap_dir / "umap_insitutype_clusters.png",
    dpi=250,
    bbox_inches="tight",
)
plt.close(fig)
print(f"Saved to {umap_dir.resolve()}")

## 7. FOV overlays with Leiden clusters

### Location of originals: DATA_Bruno → YALE Data → fov_leiden_plots

**Purpose.** Same layout as Section 2 (one figure per FOV: scatter + polygon
panels), but cells are colored by *Leiden* cluster instead of InSituType. Lets
you see where each Leiden cluster sits in physical space (per FOV).

**What each figure shows.** Left: scatter of cell centers in local pixels,
colored by Leiden (0, 1, 2, …). Right: polygon outlines with the same
Leiden colors. Useful to compare spatial distribution of data-driven
clusters (Leiden) with vendor types (Section 2).

**Output.** `fov_leiden_plots/fov_001_leiden.png` … `fov_361_leiden.png`.
Uploaded folder to **YALE Data**.

In [ ]:
# Same structure as Section 2: one dir, palette for Leiden clusters
leiden_plot_dir = Path("fov_leiden_plots")
leiden_plot_dir.mkdir(exist_ok=True)
leiden_palette = sns.color_palette("tab20", n_colors=max(len(clusters), 1))
leiden_color_lookup = {
    c: to_hex(col) for c, col in zip(clusters, leiden_palette)
}
poly = adata.uns.get("polygons")  # for right-panel outlines

In [ ]:
def make_leiden_fov_plot(fov_id: int):
    """Left: Leiden scatter; right: polygons colored by Leiden (same FOV)."""
    adata_fov = adata[adata.obs["fov"] == fov_id].copy()
    if adata_fov.n_obs == 0:
        raise ValueError(f"FOV {fov_id} has no cells.")
    x = adata_fov.obsm["spatial"][:, 0]
    y = adata_fov.obsm["spatial"][:, 1]
    cluster_series = (
        adata_fov.obs["leiden"]
        .astype("category")
        .cat.remove_unused_categories()
    )
    scatter_colors = (
        cluster_series.map(leiden_color_lookup).astype(object).fillna("#333333")
    )
    cell_to_cluster = cluster_series.to_dict()
    fig, axes = plt.subplots(1, 2, figsize=(12, 6), sharey=True)
    axes[0].scatter(x, y, s=4, c=scatter_colors)
    axes[0].set_title(f"FOV {fov_id} — Leiden scatter")
    axes[0].set_xlabel("CenterX_local_px")
    axes[0].set_ylabel("CenterY_local_px")
    axes[0].invert_yaxis()
    legend_handles = [
        mpatches.Patch(color=leiden_color_lookup[cat], label=str(cat))
        for cat in cluster_series.cat.categories
        if cat in leiden_color_lookup
    ]
    if legend_handles:
        axes[0].legend(
            handles=legend_handles,
            title="Leiden",
            loc="upper left",
            bbox_to_anchor=(1.02, 1.0),
            borderaxespad=0.0,
        )
    axes[1].set_title(f"FOV {fov_id} — Polygons")
    axes[1].set_xlabel("x_local_px")
    axes[1].set_ylabel("y_local_px")
    if poly is not None:
        poly_fov = poly[poly["fov"] == fov_id]
        for cell_label, df in poly_fov.groupby("cell"):
            cl = cell_to_cluster.get(cell_label)
            color = leiden_color_lookup.get(cl, "#777777")
            axes[1].plot(
                df["x_local_px"], df["y_local_px"],
                linewidth=0.5, color=color,
            )
    else:
        axes[1].text(0.5, 0.5, "No polygon data", ha="center", va="center")
    axes[1].invert_yaxis()
    axes[1].set_aspect("equal", adjustable="box")
    axes[0].set_aspect("equal", adjustable="box")
    fig.suptitle(f"FOV {fov_id}: Leiden clusters & polygons", fontsize=14)
    fig.tight_layout()
    return fig

In [ ]:
leiden_saved = []
for raw_fov in sorted(adata.obs["fov"].unique()):
    fov_id = int(raw_fov)
    fig = make_leiden_fov_plot(fov_id)
    out_path = leiden_plot_dir / f"fov_{fov_id:03d}_leiden.png"
    fig.savefig(out_path, dpi=250, bbox_inches="tight")
    plt.close(fig)
    leiden_saved.append(out_path)
print(f"Saved {len(leiden_saved)} Leiden FOV overlays to {leiden_plot_dir}")

## 8. Cell-type marker feature plots (curated panel)

### Location of originals: DATA_Bruno → YALE Data → feature_plots/per_celltype

**Purpose.** Visualize a curated set of marker genes for major compartments
(epithelial, immune, stroma, endothelium, fibroblasts, CAFs, etc.) on the
UMAP. Uses the *clustered* AnnData (after QC, HVG, Leiden) so embedding and
clusters are fixed.

**Data.** Load `colon_adata_clustered.h5ad` (same as Section 3 output: QC’d,
normalized, HVG subset, UMAP and Leiden computed). If you only have
`colon_adata.h5ad`, run Section 3 first and save the result as
`colon_adata_clustered.h5ad`, or point the path below to your clustered object.

**Curated panel.** A dictionary maps label names (e.g. "Epithelial",
"TCell", "Fibroblast") to lists of marker genes. We first check which genes
are absent in the dataset (HVG subset may drop some); then for every
(label, gene) pair with the gene present, we draw one UMAP colored by that
gene’s expression (blue → red, vmax=p99) and save it.

**What each plot is.** One UMAP, one gene. Title: "{label} marker: {gene}".
Same style as Section 5 (custom blue–green–yellow–red colormap). Lets you
see where each compartment’s markers are expressed in the embedding.

**Output.** `feature_plots/per_celltype/{safe_label}_{gene}_feature.png`.
Uploaded to **YALE Data**.

**Run order.** Run Section 3 first for imports and to produce the clustered
object, or use the import cell below and load `colon_adata_clustered.h5ad` directly.

In [ ]:
# Imports for Sections 8–9 (run Section 3 first for same env, or run this)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap, to_hex
import seaborn as sns
import scanpy as sc
import anndata as ad
from scipy import sparse
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")
sc.settings.verbosity = 3
print("Libraries imported successfully!")

In [ ]:
# Load clustered AnnData (QC + HVG + UMAP + Leiden); from Section 3 or saved copy
adata = ad.read_h5ad("colon_adata_clustered.h5ad")
print(adata)

In [ ]:
# Curated markers: label -> list of genes (epithelial, immune, stroma, etc.)
celltype_markers = {
    "Epithelial": [
        "KRT8", "KRT19", "KRT20", "TACSTD2", "CEACAM5", "REG4",
        "LGALS4", "SLC26A3", "SLC9A3", "AQP8", "CA1", "CA2", "FABP1",
        "GUCA2A", "MUC2", "FCGBP", "CLCA1", "AGR2", "SPINK4", "ZG16",
        "DEFA5", "DEFA6", "LYZ", "PLA2G2A", "REG3A", "CHGA", "SCG5",
        "NEUROD1", "OLFM4", "SMOC2", "CLDN1", "CLDN4", "CEACAM7",
        "DMBT1", "GCG", "RGS2",
    ],
    "Tumor_EMT_like": [
        "VIM", "ITGA5", "FN1", "ZEB1", "SNAI2", "TWIST1", "ZEB2",
        "COL1A1", "COL1A2", "TAGLN",
    ],
    "Tumor_Response_TGFB_AP1_JNK": ["TGFB2", "TGFBR2", "FOS"],
    "PanImmune": ["PTPRC", "CD74", "HLA-DRA", "HLA-DPA1"],
    "PlasmaCell": ["MZB1", "JCHAIN", "IGHG1", "IGKC", "IGHA1"],
    "TCell": [
        "CD3D", "TRAC", "TRBC1", "CD4", "IL7R", "LTB", "GZMK", "CXCR4",
    ],
    "CD4_TCell": ["CD4"],
    "NKCell": ["KLRD1", "FCGR3A", "NCAM1", "GNLY"],
    "Myeloid": ["TYROBP", "FCER1G", "AIF1", "FCGR3A"],
    "Monocyte": ["CD14", "S100A9", "LGALS3"],
    "Macrophage": [
        "C1QA", "C1QB", "C1QC", "APOE", "CTSB", "CTSD",
        "MSR1", "CD163", "FCER1G",
    ],
    "SPP1_TAM": ["SPP1", "LGALS3", "TREM2", "GPNMB", "CTSL"],
    "MastCell": ["KIT", "IL1RL1"],
    "Treg": ["CTLA4", "TIGIT", "IKZF2"],
    "Endothelial": ["PECAM1", "KDR", "ENG", "PLVAP"],
    "Lymphatic_Endo": ["PDPN", "CCL21"],
    "Pericyte": ["STEAP4", "PDGFRB", "MCAM", "NOTCH3", "ABCC9", "DES"],
    "SMC": [
        "MYH11", "ACTG2", "DES", "ACTA2", "TAGLN",
        "CNN1", "MYL9", "TPM2", "CALD1",
    ],
    "Fibroblast": [
        "VIM", "PDGFRA", "PDGFRB", "LUM", "COL1A1", "COL1A2",
        "COL3A1", "COL6A1", "COL14A1", "PI16", "COL5A1", "COL5A2",
        "LOX", "FBLN1", "FBLN2", "FN1", "VCAM1", "MGP", "SFRP2", "C3",
        "SPARC",
    ],
    "CCL8_Fibroblast": ["COL3A1"],
    "PDGFRAhi_Fibroblast": ["PDGFRA", "MMP1"],
    "Lymphoid_Stromal": ["CXCL13", "CCL19"],
    "CAF_myofibroblast": ["PDPN", "ACTA2", "TAGLN", "POSTN", "THY1", "INHBA"],
    "CAF_inflammatory": ["CXCL12", "IL6", "CCL2", "PTGS2", "CXCL8", "CSF1"],
    "Enteric_Glial": ["PLP1", "APOD"],
    "Adipose": ["FABP4", "ADIPOQ", "ADIRF"],
}

In [ ]:
# Report which genes are missing (HVG subset may not include all panel genes)
genes_in_adata = set(adata.var_names)
print("Genes absent in adata, by label:\n")
for label, genes in celltype_markers.items():
    if not genes:
        continue
    absent = [g for g in genes if g not in genes_in_adata]
    if absent:
        print(f"  {label}: {absent}")
    else:
        print(f"  {label}: (all present)")
print("\nDone checking.")

In [ ]:
# Colormap and output dir (same blue->red as Section 5)
from matplotlib.colors import LinearSegmentedColormap
colors = ["blue", "green", "yellow", "red"]
custom_cmap = LinearSegmentedColormap.from_list(
    "BlueGreenYellowRed", colors, N=256,
)
feature_plot_root = Path("feature_plots")
feature_plot_root.mkdir(exist_ok=True)
celltype_feature_dir = feature_plot_root / "per_celltype"
celltype_feature_dir.mkdir(exist_ok=True)
print(f"Feature plots will be saved to {feature_plot_root.resolve()}")

In [ ]:
# One UMAP per (label, gene); filename includes label for context
plots_created = 0
for label, genes in celltype_markers.items():
    if not genes:
        continue
    safe_label = "".join(
        c if c.isalnum() or c in "_-" else "_" for c in label
    )
    for gene in genes:
        if gene not in adata.var_names:
            continue
        sc.pl.umap(
            adata,
            color=gene,
            cmap=custom_cmap,
            size=1.5,
            vmax="p99",
            title=f"{label} marker: {gene}",
            show=False,
        )
        fig = plt.gcf()
        fig.savefig(
            celltype_feature_dir / f"{safe_label}_{gene}_feature.png",
            dpi=200,
            bbox_inches="tight",
        )
        plt.close(fig)
        plots_created += 1
print(f"Saved {plots_created} cell-type marker feature plots to {celltype_feature_dir}")

## 9. Cell-type marker feature plots with cluster breakdown

### Location of originals: DATA_Bruno → YALE Data → feature_plots/per_celltype_with_cluster_hue

**Purpose.** For the same curated markers as Section 8, produce an extended
figure: left = UMAP colored by gene expression (as before); right = horizontal
bar chart showing how the *expressing* cells (those with that gene > 0) are
distributed across Leiden clusters. Bar color matches the mean expression
level in that cluster (using the same UMAP colormap), so you see which
clusters carry low vs high expression.

**What each plot shows.**
- **Left panel:** UMAP, cells colored by the gene (blue = low, red = high,
  vmax=p99). Same as Section 8.
- **Right panel:** One horizontal bar per Leiden cluster. Bar *length* = share
  of all expressing cells that fall in that cluster (so lengths sum to 1).
  Bar *color* = mean of the UMAP colors of cells in that cluster (average
  expression level in the cluster). Colorbar on the right ties bar color to
  the expression scale. Genes with no expressing cells are skipped.

**Why this helps.** You can see at a glance which clusters dominate
expression of each marker and whether expression is strong or weak in those
clusters (bar color). Useful for assigning Leiden clusters to cell types.

**Output.** `feature_plots/per_celltype_with_cluster_hue/{safe_label}_{gene}_feature.png`.
Uploaded to **YALE Data**.

In [ ]:
# Two-panel figs: UMAP + cluster bar chart (bar = % of expressing cells, color = level)
from matplotlib.colors import Normalize
cluster_hue_dir = feature_plot_root / "per_celltype_with_cluster_hue"
cluster_hue_dir.mkdir(exist_ok=True)
cluster_col = "leiden"
clusters = sorted(adata.obs[cluster_col].astype(str).unique())
print(f"Cluster-hue feature plots -> {cluster_hue_dir.resolve()}")
plots_created = 0

In [ ]:
for label, genes in celltype_markers.items():
    if not genes:
        continue
    safe_label = "".join(
        c if c.isalnum() or c in "_-" else "_" for c in label
    )
    for gene in genes:
        if gene not in adata.var_names:
            continue
        x_gene = adata[:, gene].X
        expr = (
            x_gene.toarray().flatten()
            if sparse.issparse(x_gene)
            else np.asarray(x_gene).flatten()
        )
        expressing = expr > 0
        n_expressing = expressing.sum()
        if n_expressing == 0:
            continue
        # Same scale as UMAP for consistent bar colors
        vmax = (
            np.percentile(expr[expr > 0], 99)
            if (expr > 0).any() else 1.0
        )
        norm = Normalize(vmin=0, vmax=max(vmax, 1e-6))
        umap_colors_rgba = custom_cmap(norm(expr))
        cluster_pct = []
        cluster_mean_umap_color = []
        for cl in clusters:
            in_cl = (adata.obs[cluster_col].astype(str) == cl).values
            n_exp_in_cl = (expressing & in_cl).sum()
            cluster_pct.append(n_exp_in_cl / n_expressing)
            if in_cl.sum() > 0:
                cluster_mean_umap_color.append(
                    umap_colors_rgba[in_cl].mean(axis=0)
                )
            else:
                cluster_mean_umap_color.append(np.array(custom_cmap(0)))

In [ ]:
        fig, (ax_umap, ax_bars) = plt.subplots(
            1, 2,
            figsize=(14, max(6, len(clusters) * 0.25)),
            gridspec_kw={"width_ratios": [1.2, 0.9]},
        )
        sc.pl.umap(
            adata,
            color=gene,
            ax=ax_umap,
            cmap=custom_cmap,
            size=2,
            vmax="p99",
            show=False,
            title=f"{label} marker: {gene}",
        )
        y_pos = np.arange(len(clusters))[::-1]
        bar_widths = np.array(cluster_pct)
        bar_colors = cluster_mean_umap_color
        ax_bars.barh(
            y_pos, bar_widths, height=0.75,
            color=bar_colors, edgecolor="gray", linewidth=0.3,
        )
        ax_bars.set_yticks(y_pos)
        ax_bars.set_yticklabels(clusters, fontsize=8)
        ax_bars.set_xlim(0, 1)
        ax_bars.set_xlabel("% of expressing cells in cluster")
        ax_bars.set_title(
            "Cluster distribution (bar color = avg UMAP color in cluster)"
        )
        sm = plt.cm.ScalarMappable(cmap=custom_cmap, norm=norm)
        sm.set_array([])
        cbar = fig.colorbar(
            sm, ax=ax_bars, shrink=0.6,
            label="Expression scale (UMAP reference)",
        )
        plt.tight_layout()
        fig.savefig(
            cluster_hue_dir / f"{safe_label}_{gene}_feature.png",
            dpi=200,
            bbox_inches="tight",
        )
        plt.close(fig)
        plots_created += 1
print(f"Saved {plots_created} cluster-hue feature plots to {cluster_hue_dir}")